In [1]:
pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 60.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.10.1
    Uninstalling huggingface-hub-0.10.1:
      Successfully uninstalled huggingface-hub-0.10.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully unin

In [2]:
pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 39.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import AutoTokenizer
from optimum.onnxruntime import AutoQuantizationConfig, ORTModelForSequenceClassification, ORTQuantizer
from optimum.pipelines import pipeline

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased',  max_length=512,  padding=True, truncation=True)




In [4]:
model_id ="lilouuch/Goodreads_Books_Reviews_BERT_51"
save_dir = "bert_quantized"
model = ORTModelForSequenceClassification.from_pretrained(model_id, from_transformers=True)
qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=True)
quantizer = ORTQuantizer.from_pretrained(model)
quantizer.quantize(save_dir=save_dir, quantization_config=qconfig)
model = ORTModelForSequenceClassification.from_pretrained(save_dir)


In [5]:
onnx_classifier = pipeline("text-classification", 
                           model=model, 
                           tokenizer=tokenizer, 
                           accelerator="ort",
                           max_length=512,
                           truncation=True)


In [6]:
import pandas as pd
from datasets import Dataset
test_df = pd.read_csv('/kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv')
test_df = test_df.iloc[:50000]
test_ds = Dataset.from_pandas(test_df)

In [7]:
def predict_class(example):
    example['predicted_class'] =onnx_classifier(example['review_text']
            )
    return example


In [8]:
test_ds = test_ds.map(predict_class)

  0%|          | 0/50000 [00:00<?, ?ex/s]

In [9]:
test_set=test_ds.to_pandas()
test_set.to_csv('sub_01.csv',index=False)